<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-15 10:41:56
-------------------
qualified stocks: 88
with latest results: 83
still star stocks: 43
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  0.00
Current:  1.15 C
-------------------
Today PnL: 37.90 K (0.33%)
Current PnL: -14.37 L (-12.24%)
CY Booked + Current PnL: -2.47 L (-2.11%)
-------------------
Total profit:  3.01 L
Total loss:  -17.38 L
-------------------
Total Booked + Current PnL: 10.64 L (10.2%)
Total Booked PnL: 25.01 L (23.99%)
Curr Year Booked PnL: 11.90 L (10.34%)
Prev Year Booked PnL: 13.11 L (12.57%)
Est FTT:  1.87 C
Est FTT PnL: 71.96 L (62.55%)
-------------------
Est LTT:  2.48 C
Est LTT PnL: 1.33 C (115.66%)
Deployed:  1.04 C
Current:  1.15 C
CAGR/XIRR %: 8.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
14   BAYERCROP  4346.57  6805.0    13.15       H-SC  11.36   84069.0   
79         TCS  4164.02  4998.0     4.29       X-LC  70.45  165019.0   
72         SIS   477.00   528.0    57.98       M-SC  42.05   56942.0   
40        INFY  1436.24  2275.0     9.18       X-LC  89.77  203284.0   
47  LALPATHLAB  2630.06  3545.0   -19.93       H-SC  59.09  121686.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
14      14524.0  24809.0        1.49         20.88  29.51  56.56  12.0   
79       2622.0  29901.0       -0.04          1.61  18.12  20.03  15.0   
72     -24148.0  32821.0        1.21        -29.78  57.64  10.69 -27.0   
40      53915.0  33318.0        0.75         36.10  16.39  58.40  37.0   
47       5963.0  34291.0       -1.34          5.15  28.18  34.79   9.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
14     0.59        0.73     7.63     X40N      NTT   CHEMICALS  
79     0.09        1.43    15.81      X40      BTT          IT  
72    -0.74        0.49     1.21     X40N      NTT        MISC  
40     1.62        1.77    42.07      X40      BTT          IT  
47     0.17        1.06    38.34     X40N      NTT  HEALTHCARE

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    FTT  Dev%_PE Conviction    RSP   Current  \
1    AARTIIND   487.04  919.0    -3.60       M-SC  76.14   70733.0   
56       NHPC    82.08  115.0   180.25       M-MC  53.41  134691.0   
58      QUESS   711.66  986.0   -27.80       M-SC  68.18  140007.0   
84  UNIONBANK   123.87  163.0   -22.16       M-MC   5.11  120874.0   
81    TRIDENT    37.35   48.0    62.33       M-SC  20.45   78595.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
1      -10603.0  82743.0        3.47        -13.04  116.98  88.69 -11.0   
56      -6322.0  62874.0        2.63         -4.48   46.68  40.11 -61.0   
58     -15847.0  75926.0        2.19        -10.17   54.23  38.55 -25.0   
84     -19966.0  64462.0        2.14        -14.18   53.33  31.59 -43.0   
81     -13660.0  39966.0        2.08        -14.81   50.85  28.51 -15.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
1     -0.13        0.61     6.46       XR      NTT  CHEMICALS  
56    -0.10        1.17    12.53     XY24      NTT      POWER  
58    -0.21        1.22    36.73     XY24      NTT       MISC  
84    -0.31        1.05     5.00     XY24      NTT      BANKS  
81    -0.34        0.68     6.99       XR      NTT   TEXTILES

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
9   BAJAJFINSV  1473.47  2392.00   -17.29       X-LC  93.18  129613.0   
10  BAJFINANCE  7055.29  9150.00   -29.48       X-LC  90.91  220982.0   
34    HDFCLIFE   560.54   897.00   -18.02       H-LC  57.95  154907.0   
18      CAMPUS   294.86   393.00    -9.36       M-SC  60.23  158637.0   
22      COLPAL  2837.00  3808.06     9.05       X-MC  65.91  133380.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
9       14682.0  56965.0       -3.02         12.77  43.95  62.34   8.0   
10       2268.0  62670.0       -2.82          1.04  28.36  29.69  15.0   
34       7485.0  81001.0       -1.91          5.08  52.29  60.02   8.0   
18      -9138.0  64978.0       -1.87         -5.45  40.96  33.28 -12.0   
22      -8470.0  57020.0       -1.68         -5.97  42.75  34.23 -49.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
9      0.26        1.13    12.55      X40      BTT    FINANCE  
10     0.04        1.92    13.52      X40      BTT    FINANCE  
34     0.09        1.35    10.58      X40      BTT  INSURANCE  
18    -0.14        1.38    29.98     XY24      NTT   FOOTWEAR  
22    -0.15        1.16    11.33      X40      ATH       FMCG

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
50      LUXIND  1448.19  2442.00    28.93       M-SC   3.41  162493.0   
82  TTKPRESTIG   769.29  1149.24    13.37       X-SC  37.50  101328.0   
14   BAYERCROP  4346.57  6805.00    13.15       H-SC  11.36   84069.0   
40        INFY  1436.24  2275.00     9.18       X-LC  89.77  203284.0   
49        LTIM  5664.39  7262.39     9.16       X-LC  15.91  172640.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
50      24915.0  69498.0        1.30         18.11  42.77  68.62  11.0   
82        551.0  49225.0        0.74          0.55  48.58  49.39   0.0   
14      14524.0  24809.0        1.49         20.88  29.51  56.56  12.0   
40      53915.0  33318.0        0.75         36.10  16.39  58.40  37.0   
49       2708.0  45232.0        0.05          1.59  26.20  28.21   1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
50     0.36        1.41    59.35     X40N      NTT   TEXTILES  
82     0.01        0.88    16.39     X40N      ATH   DURABLES  
14     0.59        0.73     7.63     X40N      NTT  CHEMICALS  
40     1.62        1.77    42.07      X40      BTT         IT  
49     0.06        1.50    26.82     X200      ATH         IT

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
37         IEX     133.39     219.00    -8.88       H-SC  30.68  187981.0   
53  METROPOLIS    1399.27    3102.00     8.56       X-SC  10.23  125287.0   
29       GLAND    1463.94    3442.00    -0.50       H-SC  80.11   91060.0   
49        LTIM    5664.39    7262.39     9.16       X-LC  15.91  172640.0   
54         MRF  112863.40  150508.27    -9.68       X-MC  19.32  114417.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  XIRR  \
37      38718.0  57071.0        0.70         25.94   30.36   64.18  19.0   
53      31536.0  82552.0       -0.11         33.64   65.89  121.69  22.0   
29      12007.0  94812.0        1.04         15.19  104.12  135.12  23.0   
49       2708.0  45232.0        0.05          1.59   26.20   28.21   1.0   
54       1554.0  36087.0        0.18          1.38   31.54   33.35   NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
37     0.68        1.63    27.45       XR      NTT        MISC  
53     0.38        1.09    20.20       XR      NTT  HEALTHCARE  
29     0.13        0.79     5.01       XR      NTT      PHARMA  
49     0.06        1.50    26.82     X200      ATH          IT  
54     0.04        0.99     1.25       AR      ATH       TYRES

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  Current P/L  \
28  GICRE   335.06   583.0     7.26       M-MC  67.05  184043.0      37622.0   
40   INFY  1436.24  2275.0     9.18       X-LC  89.77  203284.0      53915.0   

    FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%   XIRR  RRR Ind  \
28  70728.0       -0.51         25.69  38.43  74.0  110.0     0.53   
40  33318.0        0.75         36.10  16.39  58.4   37.0     1.62   

    CurrAlloc%  Gained% Criteria Strategy   Category  
28        1.60    41.33     XY24      BTT  INSURANCE  
40        1.77    42.07      X40      BTT         IT

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol   AvgCost       FTT  Dev%_PE Conviction     RSP   Current  \
21   COFFEEDAY     59.14     80.00  -129.93       L-SC  100.00   55219.0   
23       DABUR    518.36    735.00    -8.65       X-MC   98.86  131006.0   
68     SBICARD    765.91   1075.00   -30.22       H-MC   97.73  171268.0   
35  HINDUNILVR   2443.56   3513.00   -14.55       X-LC   96.59  148910.0   
36      HONAUT  42646.15  58518.52   -15.86       X-MC   95.45  122228.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
21     -58330.0  98378.0       -0.00        -51.37  178.16  35.27   NaN   
23      -1176.0  56424.0        0.24         -0.89   43.07  41.79  -1.0   
68      -8721.0  81352.0       -0.80         -4.85   47.50  40.36  -4.0   
35      -5034.0  72415.0       -0.18         -3.27   48.63  43.77  -4.0   
36      -5710.0  53328.0        0.39         -4.46   43.63  37.22  97.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
21    -0.59        0.48    27.14       XR      NTT      HOTELS  
23    -0.02        1.14     4.73     XY24      BTT        FMCG  
68    -0.11        1.49     9.78     XY24      NTT     FINANCE  
35    -0.07        1.29     9.21      X40      BTT        FMCG  
36    -0.11        1.06    12.01     X40N      ATH  ELECTRICAL

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.50
1     25       40.89
2     50       71.39

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.27
X40     19.19
XR      13.59
X40N    13.41
AR       7.95
XY25     2.60
X200     1.50
SR       1.42
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M-SC    27.10
X-LC    22.69
H-SC    17.62
M-MC    10.88
X-SC     5.65
X-MC     5.09
H-MC     2.77
L-SC     2.63
M-LC     2.54
H-LC     1.35
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               13.53        -17.63   73.45
FINANCE             12.37        -14.77   63.10
IT                   8.97         -7.77   61.93
FMCG                 8.76         -6.52   45.12
MISC                 7.40         -6.45   45.91
PAINTS               5.84        -22.47   58.72
CHEMICALS            5.77         -9.06   77.88
HEALTHCARE           4.84          1.98   46.53
INSURANCE            4.31          2.37   50.30
AUTO                 3.29         -8.57   56.31
FOOTWEAR             2.98        -20.16   59.86
POWER                2.25         -9.19   52.86
TRAVEL               2.14        -24.83   75.80
TEXTILES             2.09          4.67   45.40
CERAMICS             2.01        -23.32   73.50
STEEL                1.61        -21.61  112.23
JEWELLERY            1.55        -61.99  113.21
ENGINEERING          1.46         -8.65   50.51
REFINERIES           1.39         -2.94   22.31
CEMENT               1.24        -31.84   99.67
ENTERTAINMENT        1.07        -32.63   99.32
ELECTRICAL           1.06         -4.67   43.63
TYRES                0.99          1.36   31.54
DURABLES             0.88          0.54   48.58
AC                   0.86        -28.61   99.60
PHARMA               0.79         13.19  104.12
HOTELS               0.48       -105.63  178.16

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3021966.0     30
XR        1340432.0     17
X40N       989690.0     16
X40        828058.0     12
AR         716220.0      8
SR         181968.0      2
XY25        72189.0      2
X200        45232.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
M-SC        2189635.0     27
H-SC        1729918.0     16
M-MC         938656.0     10
X-LC         872853.0     14
X-SC         460754.0      6
L-SC         294821.0      4
X-MC         243879.0      5
H-MC         149821.0      2
M-LC         116852.0      2
H-LC          81001.0      1

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1059984.0     11
M-MC       XY24       730528.0      7
H-SC       XY24       648786.0      4
X-LC       X40        625799.0      9
M-SC       XR         461916.0      6
           X40N       419219.0      6
H-SC       XR         396772.0      5
           X40N       390499.0      6
           AR         293861.0      1
X-SC       XR         186731.0      2
L-SC       XR         163656.0      2
H-MC       XY24       149821.0      2
M-MC       XR         131357.0      2
X-LC       AR         129633.0      2
M-SC       AR         122765.0      2
X-SC       SR         120455.0      1
M-LC       XY24       116852.0      2
X-MC       X40N        94348.0      2
X-SC       X40N        85624.0      2
H-LC       X40         81001.0      1
M-MC       AR          76771.0      1
L-SC       XY24        74062.0      1
X-LC       XY25        72189.0      2
X-SC       XY24        67944.0      1
M-SC       X40         64238.0      1
           SR          61513.0      1
L-SC       AR          57103.0      1
X-MC       X40         57020.0      1
           XY24        56424.0      1
X-LC       X200        45232.0      1
X-MC       AR          36087.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
